# IoU 함수 구현하기

In [ ]:
import numpy as np


def intersection_over_union(box1, box2):
    # 교집합 부분의 top left 좌표와 bottom right 좌표를 계산하세요.
    x1 = np.maximum(box1[0], box2[0])
    y1 = np.maximum(box1[1], box2[1])
    x2 = np.minimum(box1[2], box2[2])
    y2 = np.minimum(box1[3], box2[3])

    # 교집합의 넒이를 구하세요.
    intersection = np.maximum(x2 - x1, 0) * np.maximum(y2 - y1, 0)

    # 박스1의 넓이와 박스2의 넓이를 각각 구하세요.
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # 두 박스의 넒이를 더한뒤 교집합 영역 넓이를 뺴 합영역을 구하세요.
    union = box1_area + box2_area - intersection

    # 교집합의 넓이와 합영역을 이용해 IoU를 계산 후 반환하세요.

    iou = intersection / union
    
    return iou



if __name__ == "__main__":
    # 아래 두 박스는 좌상단 모서리 점과 우하단 모서점으로 표현됩니다. 
    box1 = [100, 100, 170, 180]

    box2 = [130, 140, 250, 300]
    
    # 완성한 함수를 호출하여 소수점 다섯째 자리에서 반올림하여 값을 출력합니다.
    iou = intersection_over_union(box1, box2)
    print(round(iou, 5))

# Average Precision 코드 구현

In [ ]:
import numpy


def average_precision(detection_results, ground_truth):
    ap = 0

    for i in range(detection_results.shape[0]):

        threshold = detection_results[i, 1]
        
        detected = detection_results[numpy.where(detection_results[:, 1] >= threshold)]
        TP = detected[numpy.where(detected[:, 0] == 1)]
        precision = TP.shape[0] / detected.shape[0]
        recall = TP.shape[0] / ground_truth


        # print(f"precision = {TP.shape[0]}/{detected.shape[0]}, recall = {TP.shape[0]}/{ground_truth}")
        ap += (precision * recall)

    return ap


if __name__ == "__main__":
    detection_results = numpy.array([
        [1, 0.95],
        [1, 0.91],
        [1, 0.85],
        [1, 0.81],
        [1, 0.78],
        [0, 0.68],
        [1, 0.57],
        [1, 0.45],
        [0, 0.43],
        [0, 0.13],
    ])

    # 정의한 함수를 호출한 결과를 소수점 다섯째 자리에서 반올림하여 확인합니다.
    ap = average_precision(detection_results, 15)
    print(round(ap, 4))

# NMS 함수 구현

In [ ]:
import cv2
import numpy

def intersection_over_union(box1, box2):
    # 교집합 부분의 top left 좌표와 bottom right 좌표를 계산합니다.
    x1 = numpy.maximum(box1[0], box2[0])
    y1 = numpy.maximum(box1[1], box2[1])
    x2 = numpy.minimum(box1[2], box2[2])
    y2 = numpy.minimum(box1[3], box2[3])

    # 교집합의 넒이를 구합니다.
    intersection = numpy.maximum(x2 - x1, 0) * numpy.maximum(y2 - y1, 0)

    # 박스1의 넓이와 박스2의 넓이를 각각 구합니다.
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # 두 박스의 넒이를 더한뒤 교집합 영역 넓이를 뺴면 합영역이 됩니다.
    union = box1_area + box2_area - intersection

    # iou를 계산합니다.
    iou = intersection / union
    return iou

def non_max_suppression(boxes, iou_threshold, confidence_threshold):
    # 박스의 confidence 값들을 내림차순으로 정렬하세요.
    boxes = boxes[numpy.where(boxes[:, 0] > confidence_threshold)]

    indices = numpy.argsort(-boxes, axis=0)[:, 0]
    
    # IoU를 계산하여 박스를 제거하세요.
    for i in range(boxes.shape[0]):
        for j in range(i + 1, boxes.shape[0]):
            if boxes[j, 0] == -1:
                continue
            
            iou = intersection_over_union(boxes[indices[i], 1:], boxes[indices[j], 1:])
            if iou >= iou_threshold:
                boxes[j, 0] = -1


    nms_boxes = boxes[numpy.where(boxes[:, 0] != -1)]
    # 제거된 박스를 반환하세요.
    return nms_boxes

if __name__ == "__main__":
    box1 = [0.8, 100, 100, 170, 180]
    box2 = [0.9, 130, 140, 250, 300]
    box3 = [0.6, 100, 150, 290, 170]
    box4 = [0.7, 120, 170, 160, 190]
    box5 = [0.5, 110, 110, 290, 290]
    box6 = [0.3, 240, 200, 340, 270]

    boxes = numpy.array([
        box1, box2, box3, box4, box5, box6
    ])

    nms_boxes = non_max_suppression(boxes, 0.5, 0.3)
    nms_boxes = list(nms_boxes)
    print(len(nms_boxes))